In [1]:
import pandas as pd
import requests
import numpy as np
import spacy
import art.config
import os
art.config.ART_NUMPY_DTYPE = 'str' # override dtype to str instead of float

from art.estimators.classification import BlackBoxClassifier
from art.attacks.extraction import KnockoffNets
from wex_clf import OnewexClassifier
from spacy_clf import SpacyClassifier
from sklearn.metrics import precision_recall_fscore_support

# hide ssl errors
import warnings
warnings.filterwarnings("ignore")

In [2]:
target_class_dict = {
    'fake-news': ['False', 'True'],
    'spam': ['spam', 'ham'],
    'hate-speech': ['Offensive_Language', 'Hate_Speech', 'Neither']    
}

collection_dict = {
    'fake-news': 'e54f9c0d-5b66-4c10-0000-01737d592c53',
    'spam': 'e54f9c0d-5b66-4c10-0000-01738025f3a2',
    'hate-speech': 'e54f9c0d-5b66-4c10-0000-0173802bad98'
}

def eval_clf(art_clf, texts, labels, use_case):
    preds = art_clf.predict(texts)
    if use_case == 'fake-news':
        # boolean indicators
        return precision_recall_fscore_support(labels.astype(int), np.argmax(preds, axis=1), average='macro')
    
    # string indicators
    target_labels = target_class_dict.get(use_case)
    pred_labels = [target_labels[x] for x in np.argmax(preds,axis=1)]
    return precision_recall_fscore_support(labels, pred_labels, average='macro')

session = requests.Session()
session.auth = ('admin', 'admin') # dummy credentials

for setting in ['random', 'adaptive']:
    print("Evaluating setting", setting)
    for use_case in ['fake-news', 'spam', 'hate-speech']:
        print("Evaluating use case", use_case)

        print("Loading data..")
        df = pd.read_csv(f'res/{use_case}/train.csv')
        texts = df['text'].to_numpy()
        labels = df['target'].to_numpy()
        action_ids = np.array([target_class_dict.get(use_case).index(str(x)) for x in labels]) # required for knockoff action sampling

        df_train_eval = df.sample(n=2000, random_state=212132)
        train_eval_texts = df_train_eval['text'].to_numpy()
        train_labels = df_train_eval['target'].to_numpy()

        df_test = pd.read_csv(f'res/{use_case}/test.csv')
        test_eval_texts = df['text'].to_numpy()
        test_eval_labels = df['target'].to_numpy()

        print("Loading Victim model..")
        blackbox_classifier = OnewexClassifier(
            prediction_url=f'https://localhost/api/v1/collections/{collection_dict.get(use_case)}/analyze',
            target_classes=target_class_dict.get(use_case),
            web_session=session
        )

        if os.path.exists(f'eval/{use_case}/wex_{setting}_res.csv'):
            df_stats = pd.read_csv(f'eval/{use_case}/wex_{setting}_res.csv', index_col=0)
            print("Skipping already performed baseline eval")
        else:
            print("Calculating performance baselines with blackbox..")
            p,r,f,_ = eval_clf(blackbox_classifier, train_eval_texts, train_labels, use_case)
            df_stats = pd.DataFrame(data=np.array([p,r,f,'train']).reshape((1,4)), columns=['precision', 'recall', 'fscore', 'set'], index=['baseline_r'])
            print("Train", (p,r,f))
            p,r,f,_ = eval_clf(blackbox_classifier, test_eval_texts, test_eval_labels, use_case)
            df_stats = df_stats.append(pd.DataFrame(data=np.array([p,r,f,'test']).reshape((1,4)), columns=['precision', 'recall', 'fscore', 'set'], index=['baseline_e']))
            print("Test", (p,r,f))


        print("Evaluating random strategy..")
        for nb_stolen in [1,10,100,250,500,1000,2500,5000,10000,25000,50000]:
            if f'q_{nb_stolen}_e' in df_stats.index.tolist():
                print(f"Already evaluated {nb_stolen} queries - skipping")
                if nb_stolen > texts.shape[0]:
                    # fully evaluated
                    break
                continue

            print(f"Training with {nb_stolen} queries to black box")
            knockoff = KnockoffNets(classifier = blackbox_classifier, batch_size_fit=32, batch_size_query=32, nb_stolen=nb_stolen, sampling_strategy=setting)
            np.random.seed(23435432)
            thieved_classifier = SpacyClassifier(model = spacy.load("en_core_web_sm"), target_classes=target_class_dict.get(use_case))
            knockoff.extract(x = texts, y=action_ids, thieved_classifier=thieved_classifier)

            print("Evaluating on train/test set")
            p,r,f,_ = eval_clf(thieved_classifier, train_eval_texts, train_labels, use_case)
            df_stats = df_stats.append(pd.DataFrame(data=np.array([p,r,f,'train']).reshape((1,4)), columns=['precision', 'recall', 'fscore', 'set'], index=[f'q_{nb_stolen}_r']))
            print("Train", (p,r,f))
            p,r,f,_ = eval_clf(thieved_classifier, test_eval_texts, test_eval_labels, use_case)
            df_stats = df_stats.append(pd.DataFrame(data=np.array([p,r,f,'test']).reshape((1,4)), columns=['precision', 'recall', 'fscore', 'set'], index=[f'q_{nb_stolen}_e']))
            print("Test", (p,r,f))

            print("Saving classifier")
            thieved_classifier.save(f'stolen-models-wex/{use_case}/{setting}_{nb_stolen}_queries/')
            df_stats.to_csv(f'eval/{use_case}/wex_{setting}_res.csv')

            if nb_stolen > texts.shape[0]:
                print("Evaluated max query size - Stopping")
                break


Evaluating setting random
Evaluating use case fake-news
Loading data..
Loading Victim model..
Calculating performance baselines with blackbox..
Train (0.9500277759623554, 0.9411477352809277, 0.9425551470588236)
Test (0.9527223973982473, 0.9435230777683176, 0.9453816334906979)
Evaluating random strategy..
Training with 1 queries to black box
Evaluating on train/test set
Train (0.242, 0.5, 0.32614555256064687)
Test (0.23937385103893932, 0.5, 0.3237521190431343)
Saving classifier
Training with 10 queries to black box
Evaluating on train/test set
Train (0.9729604022627278, 0.9707820808507912, 0.9714181917192485)
Test (0.9660830098556379, 0.9632402614229467, 0.9641369804726445)
Saving classifier
Training with 100 queries to black box
Evaluating on train/test set
Train (0.9372881355932203, 0.9235537190082644, 0.9251590875072313)
Test (0.941743059611477, 0.9290450899766804, 0.9311701269753341)
Saving classifier
Training with 250 queries to black box
Evaluating on train/test set
Train (0.96806

Knock-off nets: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.91it/s]


Evaluating on train/test set
Train (0.258, 0.5, 0.3403693931398417)
Test (0.2606261489610607, 0.5, 0.3426468434158489)
Saving classifier
Training with 10 queries to black box


Knock-off nets: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.78it/s]


Evaluating on train/test set
Train (0.9680602531293154, 0.9644876353385867, 0.9653647058823529)
Test (0.9627228015789493, 0.9577193983917589, 0.9590327536141029)
Saving classifier
Training with 100 queries to black box


...

Evaluating on train/test set
Train (0.6253188329334163, 0.768473081416747, 0.6589029759778163)
Test (0.6326185910193859, 0.7776450849021709, 0.6654613584695898)
Saving classifier
Training with 10000 queries to black box


Knock-off nets: 100%|████████████████████████████████████████████████████████████| 10000/10000 [12:49<00:00, 12.99it/s]


Evaluating on train/test set
Train (0.656919962074937, 0.8100822432560024, 0.6990411566061638)
Test (0.6557277579227243, 0.8046115473129808, 0.6957909482798755)
Saving classifier
Training with 25000 queries to black box


Knock-off nets: 100%|████████████████████████████████████████████████████████████| 25000/25000 [35:30<00:00, 11.74it/s]


Evaluating on train/test set
Train (0.7663632618341222, 0.8270062647973943, 0.7865072713909923)
Test (0.7872611221538746, 0.8423629908828364, 0.8063913291636542)
Saving classifier
Evaluated max query size - Stopping
